In [ ]:
# ====================================================
# Script de Colorização para Google Colab
# Baseado no DeOldify (https://github.com/jantic/DeOldify)
# Licença MIT Original: LICENSE-DeOldify
# Adaptado por: Daniel Arcanjo da Silva (2025)
# ====================================================

# ✅ 1. CLEAN INSTALLATION WITH COMPATIBLE VERSIONS
!pip uninstall -y numpy torch torchvision fastai Pillow
!pip install -q numpy==1.26.4  # Newer compatible version
!pip install -q torch==2.0.1 torchvision==0.15.2
!pip install -q fastai==1.0.61
!pip install -q ffmpeg-python yt-dlp
!pip install -q Pillow==9.5.0  # Updated version

# ✅ 2. VERIFY INSTALLATIONS
import numpy as np
import torch
print(f"\u2705 Verified installations:")
print(f"NumPy: {np.__version__} | PyTorch: {torch.__version__}")
print("NumPy-PyTorch test:", torch.from_numpy(np.array([1,2,3])))

# ✅ 3. MUST RESTART RUNTIME HERE (Runtime → Restart runtime) ⚠️

# After restart, run from here:
# ✅ 4. SETUP DEOLDIFY
!git clone https://github.com/jantic/DeOldify.git
%cd DeOldify
!mkdir -p models
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O models/ColorizeArtistic_gen.pth

# ✅ 5. IMPORTS WITH ERROR HANDLING
try:
    import numpy as np
    import torch
    from deoldify import device
    from deoldify.device_id import DeviceId
    from deoldify.visualize import *
    import warnings
    warnings.filterwarnings("ignore")
    print("\u2705 All imports successful")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    raise

# ✅ 6. GPU CONFIGURATION
device.set(device=DeviceId.GPU0)
torch.backends.cudnn.benchmark = True

# ✅ 7. COLORIZER INITIALIZATION
try:
    colorizer = get_image_colorizer(artistic=True)
    print("\u2705 Colorizer initialized")
except Exception as e:
    print(f"❌ Colorizer failed: {e}")
    raise

# ✅ 8. IMAGE UPLOAD
from google.colab import files
import os
import shutil

os.makedirs('test_images', exist_ok=True)
print("🖼️ Upload your image:")
uploaded = files.upload()

if not uploaded:
    raise ValueError("❌ No file uploaded")

uploaded_filename = next(iter(uploaded))
target_path = f'test_images/{uploaded_filename}'
shutil.move(uploaded_filename, target_path)
print(f"📁 Image saved to: {target_path}")

# ✅ 9. COLORIZATION
try:
    print("🎨 Processing...")
    result = colorizer.plot_transformed_image(
        target_path,
        render_factor=35,
        display_render_factor=True,
        figsize=(20,20)
    )
    print("✨ Colorization complete!")
except Exception as e:
    print(f"❌ Colorization failed: {e}")
    raise
